<a href="https://colab.research.google.com/github/Abubakrhegazi/Arabic-Grammar-AutoCorrection/blob/main/Arabic_GED_GEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# prompt: import from drive folder named grammar and read an m2 file

from google.colab import drive
drive.mount('/content/drive')

# file_path = '/content/drive/My Drive/grammar/QALB-Train2014.m2'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#functions dictionary
def smart_open(fname, mode = 'r'):
    if fname.endswith('.gz'):
        import gzip
        # Using max compression (9) by default seems to be slow.
        # Let's try using the fastest.
        return gzip.open(fname, mode, 1)
    else:
        return open(fname, mode)

import sys
from getopt import getopt

def load_annotation(gold_file):
    source_sentences = []
    gold_edits = []
    fgold = smart_open(gold_file, 'r')
    puffer = fgold.read()
    fgold.close()
    #puffer = puffer.decode('utf8')
    for item in paragraphs(puffer.splitlines(True)):
        item = item.splitlines(False)
        sentence = [line[2:].strip() for line in item if line.startswith('S ')]
        assert sentence != []
        annotations = {}
        for line in item[1:]:
            if line.startswith('I ') or line.startswith('S '):
                continue
            assert line.startswith('A ')
            line = line[2:]
            fields = line.split('|||')
            #print(fields)
            start_offset = int(fields[0].split()[0])
            end_offset = int(fields[0].split()[1])
            etype = fields[1]
            if etype == 'noop':
                start_offset = -1
                end_offset = -1
            corrections =  [c.strip() if c != '-NONE-' else '' for c in fields[2].split('||')]
            # NOTE: start and end are *token* offsets
            original = ' '.join(' '.join(sentence).split()[start_offset:end_offset])
            if len(fields) > 4:
              annotator = int(fields[5])
            if annotator not in annotations.keys():
                annotations[annotator] = []
            annotations[annotator].append((start_offset, end_offset, original, corrections))
        tok_offset = 0
        for this_sentence in sentence:
            tok_offset += len(this_sentence.split())
            source_sentences.append(this_sentence)
            this_edits = {}
            for annotator, annotation in annotations.items():
                this_edits[annotator] = [edit for edit in annotation if edit[0] <= tok_offset and edit[1] <= tok_offset and edit[0] >= 0 and edit[1] >= 0]
            if len(this_edits) == 0:
                this_edits[0] = []
            gold_edits.append(this_edits)
    return (source_sentences, gold_edits)
def paragraphs(lines, is_separator=lambda x : x == '\n', joiner=''.join):
    paragraph = []
    for line in lines:
        if is_separator(line):
            if paragraph:
                yield joiner(paragraph)
                paragraph = []
        else:
            paragraph.append(line)
    if paragraph:
        yield joiner(paragraph)

In [20]:
m2_list_sentence=[]
m2_list_operation=[]

m2_list_sentence.append(load_annotation('/content/drive/MyDrive/Grammar/QALB-Train2014.m2')[0])
m2_list_operation.append(load_annotation('/content/drive/MyDrive/Grammar/QALB-Test2014.m2')[1])


In [21]:
# For extracting actual corrections

def extract(operation):
  input=[]
  correct=[]

  for i in range(len(operation)):
          if (operation[i][0]!=operation[i][1]):
              input.append(operation[i][2])
              correct.append(operation[i][3][0])
  return (input , correct)

In [22]:
def make_tokens(list):
  tokens =[]
  for s in list:
      for i in s:
          tokens.append(i.split(' '))
  return tokens


In [23]:
tokens =[]
i=0
for i in m2_list_sentence:
    tokens = make_tokens(m2_list_sentence)
# print(tokens[0])
# len(tokens)

In [24]:
print("len(tokens):", len(tokens))
print("len(m2_list_operation[0]):", len(m2_list_operation[0]))


len(tokens): 19411
len(m2_list_operation[0]): 968


In [25]:
list_correction = []
list_input = []

for i in range(len(m2_list_operation[0])):  # ✅ Use the shorter list
    edits = m2_list_operation[0][i][0]
    input_, correction = extract(edits)
    list_input.append(input_)
    list_correction.append(correction)


In [26]:
for i in range(5):  # test like df.head f pandas
    print(f"Input {i}:", list_input[i])
    print(f"Correction {i}:", list_correction[i])
    print()


Input 0: ['لا زال', 'الشبيحه', 'ان', 'ارواح', 'اقل', 'كلفه', 'اذا', 'المعادله', 'المهينه', 'ان', 'هبه', 'قويه', 'واحده', 'وياخذو', 'العصابه', 'عنوه', '،', 'اننا', 'اكثر', 'اربعين', 'عام']
Correction 0: ['لازال', 'الشبيحة', 'أن', 'أرواح', 'أقل', 'كلفة', 'إذا', 'المعادلة', 'المهينة', 'أن', 'هبة', 'قوية', 'واحدة', 'ويأخذوا', 'العصابة', 'عنوة', '.', 'إننا', 'أكثر', 'أربعين', 'عاما']

Input 1: ['امريكان', 'و يا', 'اذنابهم', 'ان', 'امن', 'تستطيعوا', 'لان', 'و سترون', 'الاسد', 'الاكتاف', 'تهو', 'الامويين']
Correction 1: ['أمريكان', 'ويا', 'أذنابهم', 'أن', 'أمن', 'تستطيعون', 'لأن', 'وسترون', 'الأسد', 'الأكتاف', 'تهز', 'الأمويين']

Input 2: ['إعتقاد']
Correction 2: ['اعتقاد']

Input 3: ['راينا', 'اماكن', 'و للتقطيع', 'بالتاكيد', 'او', 'او', 'ابيا', 'واسفاه', 'و ساند', 'اذيالها']
Correction 3: ['رأينا', 'أماكن', 'وللتقطيع', 'بالتأكيد', 'أو', 'أو', 'أبيا', 'وأسفاه', 'وساند', 'أذيالها']

Input 4: ['الى', 'مشاركه', 'انت', 'الاحرار', 'لايخافون', 'لانهم', 'الانفس', 'البريئه', 'اما', 'الارانب', 'لانه'